# Data Prepration for GoodReads Dataset
## (Efficient) Loading, Cleaning, Preprocessing

### Setup

In [1]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from tqdm.notebook import tqdm


Dataset directory

In [2]:
DIR = './data'

#### Load Data
using pandas to read the JSON data - read line-by-line and convert to CSV for easier processing later on

In [3]:
in_fn_genres = 'goodreads_book_genres_initial.json.gz'

In [4]:
in_fn_books = 'goodreads_books_mystery_thriller_crime.json.gz'
in_fn_reviews = 'goodreads_reviews_mystery_thriller_crime.json.gz'
in_fn_authors = 'goodreads_book_authors.json.gz'

In [5]:
NBOOKS = 100000
NREVIEWS = 10*NBOOKS

---
read chunks of json and save to csv for easier tf.data processing

In [6]:
OUT_DIR = './data/cleaned/'
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)


In [7]:
chunk_size = 1000

book_reader = pd.read_json(os.path.join(DIR, in_fn_books), lines=True, dtype={
    "title": 'string', 
    "description": 'string', 
    "text_reviews_count": 'uint', 
    'average_rating': 'float'
}, chunksize=chunk_size)
genre_reader = pd.read_json(os.path.join(DIR, in_fn_genres), lines=True, dtype={"book_id": 'uint32'}, chunksize=chunk_size)
authors_reader = pd.read_json(os.path.join(DIR, in_fn_authors), lines=True, chunksize=chunk_size,
dtype = {
    'author': 'name'
})
review_reader = pd.read_json(os.path.join(DIR, in_fn_reviews), lines=True, chunksize=chunk_size, 
dtype = {
'book_id': 'uint32',
'rating': 'uint8',
'review_text': 'string',
})

# set the datatypes to save memory already when reading

we don't need genre data (for now)

In [8]:
# # write first chunk with header
# with open(os.path.join(OUT_DIR, "genres.csv"), 'a') as f:
#     chunk = next(genre_reader)
#     chunk.set_index("book_id", inplace=True)
#     chunk = pd.concat([chunk.drop(['genres'], axis=1), chunk['genres'].apply(pd.Series)], axis=1)
#     chunk.rename(columns={"comics, graphic": "comics_graphic", 
#                      "history, historical fiction, biography": "history_historical_fiction_biography", 
#                      "fantasy, paranormal": "fantasy_paranormal", 
#                      "mystery, thriller, crime": "mystery_thriller_crime"},
#             inplace=True)
#     chunk.to_csv(f, header=True, line_terminator='\n')    

In [9]:
# with open(os.path.join(OUT_DIR, "genres.csv"), 'a') as f:
#     no_lines = 2360656
#     for chunk in tqdm(genre_reader, total=no_lines/chunk_size):
#         chunk.set_index("book_id", inplace=True)
#         chunk = pd.concat([chunk.drop(['genres'], axis=1), chunk['genres'].apply(pd.Series)], axis=1)
#         chunk.rename(columns={"comics, graphic": "comics_graphic", 
#                     "history, historical fiction, biography": "history_historical_fiction_biography", 
#                     "fantasy, paranormal": "fantasy_paranormal", 
#                     "mystery, thriller, crime": "mystery_thriller_crime"},
#         inplace=True)

#         chunk.to_csv(f, header=False, line_terminator='\n')

---

In [10]:
def prepare_book_df(df):
    df.set_index("book_id", inplace=True)
    df.loc[:,'author_id'] = df['authors'].apply(lambda row: row[0]['author_id']) # just select the first author_id of the list of authors
    df.loc[:, 'author_id'] = df['author_id'].astype('int64')
    df = df.loc[:, ['title', 'text_reviews_count', 'average_rating', 'description', 'author_id']]
    df.loc[:,'title'] = df['title'].str.encode('utf-8')
    df.loc[:,'description'] = df['description'].replace(r'\n',' ', regex=True) 
    return df

In [11]:
# write first chunk with header
with open(os.path.join(OUT_DIR, "books.csv"), 'a') as f:
    chunk = next(book_reader)
    chunk = prepare_book_df(chunk)
    chunk.to_csv(f, header=True, line_terminator='\n')    

In [12]:
from itertools import islice

In [13]:
with open(os.path.join(OUT_DIR, "books.csv"), 'a') as f:
    for chunk in tqdm(islice(book_reader, 0, NBOOKS), total=NBOOKS/chunk_size):
        chunk = prepare_book_df(chunk)
        chunk.to_csv(f, header=False, line_terminator='\n')

  0%|          | 0/100.0 [00:00<?, ?it/s]

---

In [14]:
def prepare_review_df(df):
    df.set_index("review_id", inplace=True)
    df.drop(columns=['date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments'], inplace=True)
    df.loc[:, 'review_text'] = df['review_text'].replace(r'\n',' ', regex=True) 
    return df

In [15]:
# write first chunk with header
with open(os.path.join(OUT_DIR, "reviews.csv"), 'a') as f:
    chunk = next(review_reader)
    chunk = prepare_review_df(chunk)
    chunk.to_csv(f, header=True, line_terminator='\n')    

In [16]:
with open(os.path.join(OUT_DIR, "reviews.csv"), 'a') as f:
    for chunk in tqdm(islice(review_reader, 0, NREVIEWS), total=NREVIEWS/chunk_size):
        chunk = prepare_review_df(chunk)
        chunk.to_csv(f, header=False, line_terminator='\n')

  0%|          | 0/1000.0 [00:00<?, ?it/s]

KeyboardInterrupt: 

- genres (= books) => 2.2 Mio
- authors => 830k
- books (mystery,crime,thriller) => 220k


---
read csv and merge dataframes, save to pickle file

In [17]:
MODEL_DIR = './models/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
DIR = './data/cleaned/'

In [18]:
csv_fn_books = 'books.csv'
csv_fn_reviews = 'reviews.csv'
csv_fn_genres = 'genres.csv'

In [19]:
chunk_size = 500

book_dtype={
    "title": 'string', 
    "description": 'string', 
    "text_reviews_count": 'uint', 
    'average_rating': 'float'
}

review_dtype = {
'book_id': 'uint32',
'rating': 'uint8',
'review_text': 'string',
}

In [20]:
df_books = pd.read_csv(os.path.join(DIR, csv_fn_books), dtype=book_dtype, low_memory=True)
df_books.title = df_books.title.str.strip("b\'\"") 
df_books.set_index('book_id', inplace=True)
df_books.head()

,title,text_reviews_count,average_rating,description,author_id
book_id,,,,,
6066814,"Crowner Royal (Crowner John Mystery, #13)",15,3.93,"London, 1196. At the command of Richard the Li...",37778
33394837,The House of Memory (Pluto's Snitch #2),60,4.33,<NA>,242185
29074697,The Slaughtered Virgin of Zenopolis (Inspector...,23,3.49,"BATHS, BANKS AND ROMAN INSURRECTION Detective ...",15104629
1902202,"Dead in the Morning (Patrick Grant, #1)",8,3.30,"Gerald breezily introduced his wife, Helen, to...",190988
9671977,Aristotele e i misteri di Eleusi,3,3.54,"""I misteri di Eleusi"" e il quinto romanzo di A...",337108


In [21]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219235 entries, 6066814 to 26168430
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   title               219235 non-null  string 
 1   text_reviews_count  219235 non-null  uint64 
 2   average_rating      219235 non-null  float64
 3   description         198488 non-null  string 
 4   author_id           219235 non-null  int64  
dtypes: float64(1), int64(1), string(2), uint64(1)
memory usage: 10.0 MB


In [22]:
df_reviews = pd.read_csv(os.path.join(DIR, csv_fn_reviews), dtype=review_dtype, low_memory=True)
df_reviews.set_index('review_id', inplace=True)
df_reviews.head()

,user_id,book_id,rating,review_text
review_id,,,,
5e212a62bced17b4dbe41150e5bb9037,8842281e1d1347389f2ab93d60773d4d,6392944,3,I haven't read a fun mystery book in a while a...
2ede853b14dc4583f96cf5d120af636f,8842281e1d1347389f2ab93d60773d4d,28684704,3,"A fun, fast paced science fiction thriller. I ..."
8e4d61801907e591018bdc3442a9cf2b,8842281e1d1347389f2ab93d60773d4d,32283133,0,http://www.telegraph.co.uk/culture/10...
022bb6daffa49adc27f6b20b6ebeb37d,8842281e1d1347389f2ab93d60773d4d,17860739,4,An amazing and unique creation: JJ Abrams and ...
0e317947e1fd341f573192111bb2921d,8842281e1d1347389f2ab93d60773d4d,8694005,3,The Name of the Rose is a thrilling Dan Brown-...


In [23]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1025000 entries, 5e212a62bced17b4dbe41150e5bb9037 to e25d87abdb0bddcda1193034e54f819e
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   user_id      1025000 non-null  object
 1   book_id      1025000 non-null  uint32
 2   rating       1025000 non-null  uint8 
 3   review_text  1024749 non-null  string
dtypes: object(1), string(1), uint32(1), uint8(1)
memory usage: 28.3+ MB


TODO: Maybe the Data should be indexed/grouped by BookID, not ReviewID

In [24]:
df_join = pd.merge(df_reviews, df_books, left_on="book_id", right_index=True)
df_join.head()

,user_id,book_id,rating,review_text,title,text_reviews_count,average_rating,description,author_id
review_id,,,,,,,,,
5e212a62bced17b4dbe41150e5bb9037,8842281e1d1347389f2ab93d60773d4d,6392944,3,I haven't read a fun mystery book in a while a...,"The Murder on the Links (Hercule Poirot, #2)",42,3.8,"On a French golf course, a millionaire is foun...",123715
ee9cf5e49795718b9bb086c1cde23116,24d87e891f6f60ae101c1de158cbe672,6392944,4,"A fun, fast read! I just adore reading books w...","The Murder on the Links (Hercule Poirot, #2)",42,3.8,"On a French golf course, a millionaire is foun...",123715
b5a6af59482018d0814c373723080adb,d083ade0fb4502508b1f3ce59bf1c7db,6392944,4,"Full of twists and turns, way better than the ...","The Murder on the Links (Hercule Poirot, #2)",42,3.8,"On a French golf course, a millionaire is foun...",123715
2ede853b14dc4583f96cf5d120af636f,8842281e1d1347389f2ab93d60773d4d,28684704,3,"A fun, fast paced science fiction thriller. I ...",Dark Matter,1026,4.1,"""Are you happy with your life?"" Those are the ...",442240
31db9238ec11ca671a65a50643f952b7,01ec1a320ffded6b2dd47833f2c8e4fb,28684704,5,"Mind-bending and completely unique, take a cha...",Dark Matter,1026,4.1,"""Are you happy with your life?"" Those are the ...",442240


In [25]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1025000 entries, 5e212a62bced17b4dbe41150e5bb9037 to e25d87abdb0bddcda1193034e54f819e
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   user_id             1025000 non-null  object 
 1   book_id             1025000 non-null  uint32 
 2   rating              1025000 non-null  uint8  
 3   review_text         1024749 non-null  string 
 4   title               1025000 non-null  string 
 5   text_reviews_count  1025000 non-null  uint64 
 6   average_rating      1025000 non-null  float64
 7   description         981948 non-null   string 
 8   author_id           1025000 non-null  int64  
dtypes: float64(1), int64(1), object(1), string(3), uint32(1), uint64(1), uint8(1)
memory usage: 67.4+ MB


In [26]:
df_join.sample(20)

,user_id,book_id,rating,review_text,title,text_reviews_count,average_rating,description,author_id
review_id,,,,,,,,,
174f91ef13dfaec90ed635346ee93658,a6cbb4292887c5c877afdc52e512c063,21853662,2,Not for me. Did not like either of the main ch...,All the Old Knives,516,3.68,"Six years ago in Vienna, terrorists took over ...",327983
6b3e45be6a75eb214f37cfe56fc675a0,fd654ef8391d74fe4bf074e95dcc3142,18039271,3,"El libro es entretenido, sin mas. Puede que yo...",El pantano de las mariposas,79,4.16,"El pantano de las mariposas, de Federico Axat,...",4806021
58c56b18a38ad05ab6d3114f21b011de,106480d58e346b0c3bee1e92ed17937a,22895696,4,Wow! Has this guy written before? Because if s...,The Whites,1024,3.62,The electrifying tale of a New York City polic...,8436083
ba4762f09fe6bfce5c1ff3b27991eb61,ff9635a0ad6dfafc19d9fd01b1f4237c,13018466,4,Some of the stories were good and have encoura...,Mystery Writers of America Presents Vengeance,78,3.65,When a different kind of justice is needed---s...,5091
be40a777025bd4077062fe762cabf0fa,8640f1f17f358f5927bf9404dc8e17eb,40440,5,This is now one of my favorite books. As an av...,The Thirteenth Tale,17104,3.95,All children mythologize their birth...So begi...,22665
c2cb1c6d0c73f51b5266ab2f3cca2be8,15983dde87ec286c4d20c8d5a086d761,22571700,2,Not a mystery. The only redeeming quality for ...,Aunt Dimity and the Summer King (An Aunt Dimit...,216,3.82,Cozy mystery lovers' favorite paranormal sleut...,140848
d5cb8848f680f0cea040dd4d530d3264,54abd0c34a9fd3c4b1f8e185d1672a27,20559676,4,"4 Stars Ohohohoo, really liked this one!","Killing Sarai (In the Company of Killers, #1)",901,4.17,"From New York Times, USA Today& Wall Street Jo...",5437976
6b092ce95adac9a7f9695e8422bb8a54,ce57730e63f6c6119a7fb7ff72a9247c,2197227,5,"I love these authors writing together, and thi...","Cemetery Dance (Pendergast, #9)",844,4.00,Pendergast-the world's most enigmatic FBI Spec...,12577
68be74a46b9d6f37e274b97c1257253e,49df10696d0fa6dd8f5328083c88504c,18801468,2,Almost 2/3 of this is a preview of Gods of Guilt.,"Switchblade (Harry Bosch, #18.5; Harry Bosch U...",47,3.66,LAPD Detective Harry Bosch seeks justice for t...,12470


before:

Index: 22726 entries, 0e317947e1fd341f573192111bb2921d to 921812c9edc173c6d12e000723b9e667

after:

Index: 735000 entries, 5e212a62bced17b4dbe41150e5bb9037 to a91f52d910b3988d02d9c72d4731de3c
(= number of reviews)

after dropNA():
703594 


In [27]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1025000 entries, 5e212a62bced17b4dbe41150e5bb9037 to e25d87abdb0bddcda1193034e54f819e
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   user_id             1025000 non-null  object 
 1   book_id             1025000 non-null  uint32 
 2   rating              1025000 non-null  uint8  
 3   review_text         1024749 non-null  string 
 4   title               1025000 non-null  string 
 5   text_reviews_count  1025000 non-null  uint64 
 6   average_rating      1025000 non-null  float64
 7   description         981948 non-null   string 
 8   author_id           1025000 non-null  int64  
dtypes: float64(1), int64(1), object(1), string(3), uint32(1), uint64(1), uint8(1)
memory usage: 67.4+ MB


In [28]:
df_join.dropna(inplace=True)

In [29]:
df_join.describe()

,book_id,rating,text_reviews_count,average_rating,author_id
count,9.817050e+05,981705.000000,981705.000000,981705.000000,9.817050e+05
mean,1.342523e+07,3.677918,2699.708815,3.869557,2.079440e+06
std,1.059178e+07,1.194058,9414.000269,0.278795,3.486725e+06
min,1.640000e+02,0.000000,0.000000,0.000000,1.400000e+01
25%,1.979852e+06,3.000000,37.000000,3.720000,1.502100e+04
50%,1.331631e+07,4.000000,213.000000,3.890000,1.641870e+05
75%,2.255727e+07,5.000000,1136.000000,4.050000,3.111153e+06
max,3.646717e+07,5.000000,78438.000000,5.000000,1.733341e+07


In [30]:
data = df_join

In [31]:
data.sample(5)

,user_id,book_id,rating,review_text,title,text_reviews_count,average_rating,description,author_id
review_id,,,,,,,,,
fcd3bccf16cac0ac6f48e1787e6227de,99714b686dbf18a831c3d5a033c9603a,25459160,3,"The book is written in a relentless way, givin...",A Rapariga no Comboio,288,3.88,"Todos os dias, Rachel apanha o comboio... No c...",1063732
84ef7a4af30cdce944020070273efdf1,fb96a1b2a291b4a2db84b4505583790e,25561727,5,No one is quite sure what the ladies in the St...,You Can't Get Blood Out of Shag Carpet (A Stud...,45,4.11,"Wanda Jean Milton discovers her husband, local...",10214416
7d0a5109c7fc82cc427bdd5eb763c27d,f5ba5f289c2737c923131689c03a9966,22522120,5,"""Debt and Defiance"" by Honor Amelia Dawson is ...",Debt and Defiance,3,4.85,Snappy Leanne struggles with a recent turn of ...,8507303
294b8154ef3fe1e8b8e2cd0871a82a92,052fda43394eff719e77ab964ecf5cb6,115447,3,Somehow the reviews had given me an idea that ...,The Arrangement,29,3.23,Alison Fairmont Villard wakes in a hospital be...,2514
057523c62ab59954aa56f8f173047e02,0272428f61f7901dd79935e2bf2d47a4,24310266,5,E alla fine (piu o meno) fu tutto chiaro Ho ...,Dieci piccoli indiani,1,4.23,"""La mensola dl caminetto era priva di ornament...",123715


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 981705 entries, 5e212a62bced17b4dbe41150e5bb9037 to e25d87abdb0bddcda1193034e54f819e
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             981705 non-null  object 
 1   book_id             981705 non-null  uint32 
 2   rating              981705 non-null  uint8  
 3   review_text         981705 non-null  string 
 4   title               981705 non-null  string 
 5   text_reviews_count  981705 non-null  uint64 
 6   average_rating      981705 non-null  float64
 7   description         981705 non-null  string 
 8   author_id           981705 non-null  int64  
dtypes: float64(1), int64(1), object(1), string(3), uint32(1), uint64(1), uint8(1)
memory usage: 64.6+ MB


In [33]:
data.to_pickle(os.path.join(DIR, "joined_df.pkl"))